In [1]:
pip install transformers datasets peft accelerate bitsandbytes trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("raultherockstar1/nyayasathi")


README.md:   0%|          | 0.00/366 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/227k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/508 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

model_name = "Equall/Saul-7B-Instruct-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def format_chat(example):
    prompt = [
        {"role": "user", "content": f"{example['instruction']} {example['input']}"},
        {"role": "assistant", "content": example['output']}
    ]
    formatted = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    return {"text": formatted}

dataset = dataset.map(format_chat)


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

In [4]:
dataset


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 508
    })
})

In [5]:
import torch

In [6]:
from transformers import AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [14]:
# Fix: Add pad_token if missing
tokenizer.pad_token = tokenizer.eos_token

# Tokenize and prepare labels
def tokenize_and_format(example):
    result = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=1024,
    )
    result["labels"] = result["input_ids"].copy()
    return result

# Apply to dataset
tokenized_dataset = dataset.map(tokenize_and_format, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/508 [00:00<?, ? examples/s]

In [17]:
def build_prompt(example):
    prompt = f"### Instruction:\n{example['instruction']}\n"
    if example['input']:
        prompt += f"### Input:\n{example['input']}\n"
    prompt += f"### Response:\n{example['output']}"
    return {"text": prompt}


In [18]:
dataset = dataset.map(build_prompt)


Map:   0%|          | 0/508 [00:00<?, ? examples/s]

In [20]:
def tokenize_and_format(example):
    result = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=1024,
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize_and_format, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/508 [00:00<?, ? examples/s]

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./qlora-saul-nyayasathi",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
    tokenizer=tokenizer
)



<ipython-input-21-20f4f3764bc4>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended,

Step,Training Loss
10,1.001000
20,0.747300
30,0.671800
40,0.667800
50,0.627700
60,0.636300
70,0.545100
80,0.543500
90,0.539700
100,0.543200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=189, training_loss=0.5673484953622969, metrics={'train_runtime': 5157.7147, 'train_samples_per_second': 0.295, 'train_steps_per_second': 0.037, 'total_flos': 6.573808572746957e+16, 'train_loss': 0.5673484953622969, 'epoch': 2.9606299212598426})

In [23]:
trainer.save_model("./qlora-saul-nyayasathi-final")
tokenizer.save_pretrained("./qlora-saul-nyayasathi-final")


('./qlora-saul-nyayasathi-final/tokenizer_config.json',
 './qlora-saul-nyayasathi-final/special_tokens_map.json',
 './qlora-saul-nyayasathi-final/tokenizer.model',
 './qlora-saul-nyayasathi-final/added_tokens.json',
 './qlora-saul-nyayasathi-final/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # Login with your Hugging Face account token

trainer.push_to_hub("anmolpokhriyal/qlora-saul-nyayasathi")
tokenizer.push_to_hub("anmolpokhriyal/qlora-saul-nyayasathi")
